In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [6]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyAt


In [7]:
openai = OpenAI()

In [8]:
google.generativeai.configure()

In [9]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [10]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [11]:
completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the graph?

Because it was too *scatter*-brained and couldn't find the right *correlation*!


In [14]:
completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the spreadsheet?

Because she thought he was too "cell-fish" and couldn’t commit to a single column!


In [12]:
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the equal sign so humble?

Because it knew it wasn't less than or greater than anyone else!



In [13]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Why was the equal sign so humble?\n\nBecause it knew it wasn't less than or greater than anyone else!\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.3420204925537109
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 22,
        "candidates_token_count": 25,
        "total_token_count": 47
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [15]:
prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [16]:
stream = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)


# How to Decide if a Business Problem is Suitable for an LLM Solution

Large Language Models (LLMs) like GPT-4 can be powerful tools, but they are not ideal for every business problem. To determine if an LLM solution is appropriate, consider the following criteria:

## 1. Nature of the Problem
- **Text-Centric:** The problem involves natural language understanding, generation, summarization, translation, or extraction.
- **Conversational or Interactive:** Use cases like chatbots, virtual assistants, or customer support.
- **Knowledge-Intensive:** Requires synthesizing information from large text corpora or generating explanations.
- **Creative Tasks:** Content creation, marketing copy, or brainstorming ideas.

## 2. Data Availability and Format
- **Unstructured Text Data:** LLMs excel with unstructured text rather than structured data like spreadsheets or databases.
- **Sufficient Context:** The input data must provide enough context for the model to understand and generate meaningful output.
- **Privacy and Compliance:** Ensure data used respects privacy and regulatory constraints, as LLMs may have limitations in handling sensitive information securely.

## 3. Desired Output
- **Natural Language Outputs:** Responses, summaries, code generation, or text transformations.
- **Flexibility over Precision:** Situations where approximate or creative answers are acceptable over exact numeric predictions.

## 4. Complexity and Specificity
- **General vs. Specialized Knowledge:** LLMs are good at general knowledge and language tasks but may struggle with highly specialized or domain-specific problems without fine-tuning.
- **Rule-Based vs. Generative:** Problems requiring strict logic or deterministic outputs may not suit LLMs well.

## 5. Scalability and Integration
- **API or Cloud Access:** LLMs typically require cloud APIs; consider latency, cost, and integration complexity.
- **Real-Time Needs:** Evaluate if the LLM’s response time fits operational requirements.

## 6. Cost Considerations
- **Budget for Usage:** LLMs can be costly at scale; weigh this against the business value.
- **Maintenance and Updates:** Consider ongoing costs for model updates, monitoring, and compliance.

---

## Summary Checklist

| Criteria                         | Suitable for LLM?                         |
|---------------------------------|-----------------------------------------|
| Problem involves natural language| Yes                                     |
| Requires structured numeric data | No                                      |
| Needs creative or flexible output| Yes                                     |
| Demands strict, rule-based logic | No                                      |
| Sensitive data handling          | Caution needed                          |
| Real-time critical responses    | Depends on latency tolerance             |
| Budget supports API usage       | Yes                                     |

---

**In conclusion:**  
If your business problem involves understanding or generating human language, is not strictly rule-based, and you have access to appropriate textual data and budget, an LLM solution is likely suitable. Otherwise, consider alternative AI or traditional software approaches.

